In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import copy

In [3]:
def smooth_image_with_gaussian(image ,kernel_size = 7):
  out = cv2.GaussianBlur(image, (kernel_size, kernel_size), 0)
  return out
 

In [4]:
def get_edges_with_canny(image,low = 100,high = 150):
  out = cv2.Canny(image, low, high)
  return out
  

In [5]:
def create_mask_for_image(image_color , edges):
  mask = np.zeros_like(edges)

  three_points = np.array([[(0, image_color.shape[0]), ((image_color.shape[1]/2), (image_color.shape[0]/2) ), (image_color.shape[1], image_color.shape[0])]]).astype("int32")
  cv2.fillPoly(mask, three_points, 255)
  
  out = cv2.bitwise_and(edges, mask)
  return out,mask

In [6]:
def hough_lines(image, rho = 3, theta = np.pi / 180, threshold = 20, min_line_len = 150, max_line_gap = 37):
  lines = cv2.HoughLinesP(image, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
  return lines


In [7]:
def draw_lines_all(img, lines, color=[255, 0, 0], thickness=7):
    for line in lines:
        for x1, y1, x2, y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)

In [8]:
def merge_lines_with_mean(image ,  lines,befor_lines,color=[255, 0, 0], thickness=7,down_road = 540 , top_road = 320 ):
    down_p = []
    up_p = []
    down_n = []
    up_n = []
    down_slope =0.5
    up_slope = 0.8
    for line in lines:
        for x1,y1,x2,y2 in line:
            slope = ((y2-y1)/(x2-x1))
            b = y1 - slope*x1
            if slope > down_slope and slope < up_slope:
                down_p.append((down_road - b)/slope)
                up_p.append((top_road - b)/slope)         
            elif slope < (-1 * down_slope) and slope > (-1 * up_slope):
                down_n.append((down_road - b)/slope)
                up_n.append((top_road - b)/slope)

    output = np.array([[int(np.mean(down_p)), int(np.mean(down_road)), int(np.mean(up_p)), int(np.mean(top_road))], [int(np.mean(down_n)), int(np.mean(down_road)), int(np.mean(up_n)), int(np.mean(top_road))]])
    for i in range(len(output)):
      if befor_lines is not None:
        output[i, 0] = (output[i, 0] + befor_lines[i,0])//2
        output[i, 2]= (output[i, 2]+ befor_lines[i,2])//2
        cv2.line(image, (output[i, 0], output[i, 1]), (output[i, 2], output[i, 3]), color, thickness)
      else:
        cv2.line(image, (output[i, 0] , output[i, 1]), (output[i, 2], output[i, 3]), color, thickness)

    return output

In [9]:
def prdeict_line(image,lines_mean_1 , lines_mean_2,color=[0, 255, 0], thickness=10):
  new_line = copy.deepcopy(lines_mean_1)
  
  for i in range(len(lines_mean_1)):
        new_line[i, 0]= lines_mean_1[i, 0] + (lines_mean_1[i, 0] - lines_mean_2[i, 0] )//5
        new_line[i, 2]=lines_mean_1[i, 2] + (lines_mean_1[i, 2] - lines_mean_2[i, 2] )//5
        cv2.line(image,(new_line[i,0] ,new_line[i, 1]), (new_line[i, 2],new_line[i, 3] ), color, thickness)
  return new_line
  

In [14]:
def find_road(image_color, lines_mean_1,lines_mean_2,show_middle_outputs=False,rho = 3,
              theta = np.pi / 180, threshold = 20, min_line_len = 40,
              max_line_gap = 35 , p1 =540 , p2 = 320):
  grayscale = cv2.cvtColor(image_color, cv2.COLOR_RGB2GRAY)

  blur = smooth_image_with_gaussian(grayscale,kernel_size = 3)
  if show_middle_outputs:
    plt.imshow(blur,cmap="gray")
    plt.show()

  edges = get_edges_with_canny(blur,low = 50,high = 150)
  if show_middle_outputs:
    plt.imshow(edges,cmap="gray")
    plt.show()

  masked_edges,mask = create_mask_for_image(image_color , edges)
  if show_middle_outputs:
    plt.imshow(mask,cmap="gray")
    plt.show()
    plt.imshow(masked_edges,cmap="gray")
    plt.show()

  output = copy.deepcopy(image_color)
  output_2 = copy.deepcopy(image_color)
  lines_all = hough_lines(masked_edges, rho = rho, theta =theta,
                          threshold = threshold,
                          min_line_len = min_line_len,
                          max_line_gap = max_line_gap)
  try :
    draw_lines_all(output,lines_all,color=[0, 255, 0], thickness=10)
    if show_middle_outputs:
      plt.imshow(output)
      plt.show()

    out_lines = merge_lines_with_mean(output_2 ,lines_all,lines_mean_1,color=[0, 255, 0],
                          thickness=3,down_road = p1 , top_road = p2)
    lines_mean_2 = lines_mean_1
    lines_mean_1 = out_lines
  except:
    out_lines = prdeict_line(output_2,lines_mean_1 , lines_mean_2,color=[255, 0, 0] , thickness=3)
    lines_mean_2 = lines_mean_1
    lines_mean_1 = out_lines
  if show_middle_outputs:
    plt.imshow(output_2)
    plt.show()
  return output_2,lines_mean_1,lines_mean_2

In [15]:
cap = cv2.VideoCapture('/content/drive/MyDrive/MV/HW2/Inputs/vid1.mp4')
outputs = []
if (cap.isOpened()== False): 
  print("Error opening video stream or file")
i=1

frameSize = (960, 540)
out = cv2.VideoWriter('output_video_01.avi',cv2.VideoWriter_fourcc(*'DIVX'), 25, frameSize)
ln1 = None
ln2 = None

while(cap.isOpened()):
  ret, frame = cap.read()
  if ret == True:
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    print(i)
    i+=1

    output,ln1 , ln2 = find_road(frame, ln1 , ln2 ,show_middle_outputs=False,rho = 3, theta = np.pi / 180, threshold = 35, min_line_len = 40, max_line_gap = 17 , p1 =540 , p2 = 320)
    output = cv2.cvtColor(output, cv2.COLOR_RGB2BGR)
    outputs.append(output)
    out.write(output)

  else: 
    break

out.release()
cap.release()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221


In [ ]:
cap = cv2.VideoCapture('/content/drive/MyDrive/MV/HW2/Inputs/vid2.mp4')
outputs = []
if (cap.isOpened()== False): 
  print("Error opening video stream or file")
i=1

frameSize = (1280, 720)
out = cv2.VideoWriter('output_video_02.avi',cv2.VideoWriter_fourcc(*'DIVX'), 25, frameSize)

ln1 = None
ln2 = None

while(cap.isOpened()):
  ret, frame = cap.read()
  if ret == True:
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    # print(i)
    i+=1
    output , ln1,ln2 = find_road(frame , ln1 , ln2, show_middle_outputs=False,
                      rho = 3, theta = np.pi / 180,
                      threshold = 45, min_line_len = 40,
                      max_line_gap = 15 , p1 =720 , p2 = 470)
  
    output = cv2.cvtColor(output, cv2.COLOR_RGB2BGR)
    outputs.append(output)
    out.write(output)
  
  else: 
    break

out.release()
cap.release()